## Using Agents in LlamaIndex
### Environement Setup

In [1]:
!pip install -q -r requirements.txt

In [2]:
from my_config import MyConfig
my_config = MyConfig()

`LlamaIndex` supports three main types of reasoning agents:

1. `Function Calling Agents` - These work with AI models that can call specific functions.
2. `ReAct Agents` - These can work with any AI that does chat or text endpoint and deal with complex reasoning tasks.
3. `Advanced Custom Agents` - These use more complex methods to deal with more complex tasks and workflows.

### Basic Agents and States

#### Getting My LLM

In [4]:
from my_utils import RunPodLLamaAgentQwenLLM
model_id="Qwen/Qwen2.5-Coder-7B-Instruct"
vllm_api_base = f"https://{my_config.VLLM_LLM_INFERENCE_NODE_IP}-8000.proxy.runpod.net/v1/chat/completions"
llm = RunPodLLamaAgentQwenLLM(api_url=vllm_api_base, overriden_model_name=model_id)

/Users/potter/Documents/MLArena/MLOps-Platform-Project/Hands-On/hfagnt-2-2-the-llamaindex-framework/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent
from llama_index.core.tools import FunctionTool

# define sample Tool -- type annotations, function names, and docstrings, are all included in parsed schemas!
def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the resulting integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Adds two integers and returns the resulting integer"""
    return a + b

# initialize agent
agent = AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply), FunctionTool.from_defaults(add)],
    llm=llm
)

# workflow = FunctionAgent(
#     agent=[multiply, add],
#     llm=llm,
#     system_prompt="You are an agent that can perform basic mathematical operations using tools.",
# )

**Agents are stateless by default**, add remembering past interactions is opt-in using a `Context` object This might be useful if you want to use an agent that needs to remember previous interactions, like a chatbot that maintains context across multiple messages or a task manager that needs to track progress over time.

In [6]:
# stateless
response = await agent.run("What is 2 times 2?")
response
# response = await workflow.run(user_msg="What is 20+(2*4)?")
# print(response)

Called with args: kwargs: {}
Model response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 2, "b": 2}
Called with args: kwargs: {}
Model response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 2 times 2 is 4.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='2 times 2 is 4.')]), tool_calls=[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 2, 'b': 2}, tool_id='a3408c1a-4699-4f01-bbc1-faa872c80d25', tool_output=ToolOutput(content='4', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False)], raw=None, current_agent_name='Agent')

In [7]:
# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)

Called with args: kwargs: {}
Model response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Answer: Hello, Bob! How can I assist you today?
Called with args: kwargs: {}
Model response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Your name was Bob.
